<a href="https://colab.research.google.com/github/zmgy107/Pytorch-Learning-Notes/blob/main/Build_the_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

torch.nn namespace provides all the building blocks your own neural network.Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [10]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

# Get Device for Training

train our model on a hardware accelerator like the GPU.

In [11]:
# check if torch.cuda is available,else continue to use the CPU
device="cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Define the Class

We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

In [12]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten=nn.Flatten()
    self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self,x):
    x=self.flatten(x)
    logits=self.linear_relu_stack(x)
    return logits

We create an instance of NeuralNetwork, and move it to the device, and print its structure.

In [13]:
# 建完模型后要把它移到cuda上
model=NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

In [14]:
X=torch.rand(1,28,28,device=device)
logits=model(X)
pred_probab=nn.Softmax(dim=1)(logits)
y_pred=pred_probab.argmax(1)
print(f"Predicted class:{y_pred}")

Predicted class:tensor([0], device='cuda:0')


# Model Layers

break down the layers in the FashionMNIST model

In [15]:
# take a sample minibatch of 3 images of size 28x28
input_image=torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten

initialize the nn.Flayyen layer to convert each 2D 28x28 image into a contiguous array of 728 pixel values

In [16]:
flatten=nn.Flatten()
flat_image=flatten(input_image)
print(flat_image.size())                   

torch.Size([3, 784])


nn.Linear

apply a linear transformation on the input using its stored weights and bias

In [17]:
layer1=nn.Linear(in_features=28*28,out_features=20)
hidden1=layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU

Non-linear activation:create the complex mappings between the model’s inputs and outputs.And helping neural networks learn a wide variety of phenomena.

ReLU is one of non-linear activations

In [18]:
print(f"Before ReLU:{hidden1}\n\n")
hidden1=nn.ReLU()(hidden1)
# hidden1=nn.ReLU(hidden1) ReLU(inplace=False)Layer中间为布尔型，输入input要在后面再加一个括号放变量
print(f"After ReLU:{hidden1}\n\n")

Before ReLU:tensor([[ 0.1319,  0.0690,  0.2079, -0.0240,  0.5390, -0.2925,  0.1330,  0.4809,
          0.7172, -0.0021,  0.2440,  0.2764, -0.0707, -0.0601,  0.1760,  0.2183,
          0.3873, -0.1093, -0.1698, -0.6499],
        [ 0.1927,  0.2131,  0.1202,  0.1268,  0.4611,  0.0626,  0.1407,  0.1688,
          0.6645,  0.0530,  0.0447,  0.0931, -0.0702,  0.2875, -0.0618,  0.0325,
          0.2941, -0.2394,  0.0140, -0.4419],
        [ 0.1803, -0.0505,  0.1455, -0.1654,  0.1768, -0.0112,  0.0057,  0.4307,
          0.6630, -0.0902,  0.1288,  0.2438, -0.2992, -0.1112, -0.1670,  0.1760,
          0.4547,  0.0066, -0.1309, -0.7319]], grad_fn=<AddmmBackward0>)


After ReLU:tensor([[0.1319, 0.0690, 0.2079, 0.0000, 0.5390, 0.0000, 0.1330, 0.4809, 0.7172,
         0.0000, 0.2440, 0.2764, 0.0000, 0.0000, 0.1760, 0.2183, 0.3873, 0.0000,
         0.0000, 0.0000],
        [0.1927, 0.2131, 0.1202, 0.1268, 0.4611, 0.0626, 0.1407, 0.1688, 0.6645,
         0.0530, 0.0447, 0.0931, 0.0000, 0.2875, 0.0000

nn.Sequential

nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [19]:
seq_medules=nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)# 注意输入输入参量个数
)
input_image=torch.rand(3,28,28)
logits=seq_medules(input_image)

nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [20]:
softmax=nn.Softmax(dim=1)
pred_probab=softmax(logits)

# Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [22]:
print(f"Model structure:{model}\n\n")

for name,param in model.named_parameters():
  print(f"Layer:{name}|Size:{param.size()}|Values:{param[:2]}\n")

Model structure:NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer:linear_relu_stack.0.weight|Size:torch.Size([512, 784])|Values:tensor([[-0.0253,  0.0141,  0.0333,  ..., -0.0183,  0.0181, -0.0307],
        [ 0.0336,  0.0227,  0.0085,  ..., -0.0292,  0.0346, -0.0030]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer:linear_relu_stack.0.bias|Size:torch.Size([512])|Values:tensor([-0.0214,  0.0233], device='cuda:0', grad_fn=<SliceBackward0>)

Layer:linear_relu_stack.2.weight|Size:torch.Size([512, 512])|Values:tensor([[ 0.0284, -0.0289, -0.0316,  ..., -0.0117,  0.0099, -0.0254],
        [-0.0195,  0.0402,  0.0050,  ...,  0.0435,  0.0105,  0.0260]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer:linea